
# 📊 เครื่องมือสรุปและแปลสไลด์ PowerPoint

สคริปต์นี้ใช้สำหรับดึงข้อความจากไฟล์ PowerPoint (.pptx) แล้วสรุปเนื้อหาของแต่ละสไลด์เป็นประโยคภาษาอังกฤษสั้น ๆ  
จากนั้นแปลเป็นภาษาไทยโดยอัตโนมัติ

## 🔧 ไลบรารีที่ใช้หลัก ๆ
- `python-pptx`: ใช้ในการดึงข้อความจากสไลด์ PowerPoint
- `openai`: ใช้ GPT ในการสรุปเนื้อหาเป็นภาษาอังกฤษ
- `google.cloud.translate`: ใช้ Google Translate API แปลเป็นภาษาไทย
- `dotenv`: จัดการตัวแปรสภาพแวดล้อม เช่น API Key
- `tqdm`: แสดงแถบสถานะระหว่างการทำงาน

## 📌 ลำดับการทำงาน
1. อ่านไฟล์ .pptx และดึงข้อความจากแต่ละสไลด์
2. ใช้ GPT สรุปข้อความของแต่ละสไลด์ให้เป็นประโยคเดียวในภาษาอังกฤษ
3. แปลประโยคที่ได้เป็นภาษาไทยด้วย Google Translate API
4. สามารถเขียนผลลัพธ์ลงไฟล์ข้อความได้

In [1]:
import os, time
from pathlib import Path
from pptx import Presentation
from tqdm import tqdm
import openai
from google.cloud import translate_v2 as translate
from dotenv import load_dotenv

# 📤 .pptx ไฟล์จาก PowerPoint แล้วดึงข้อความจากแต่ละสไลด์
def extract_slide_text(pptx_path):
    from pptx import Presentation

    prs = Presentation(pptx_path)
    pages = []
    for i, slide in enumerate(prs.slides, start=1):
        items = []
        for shape in slide.shapes:
            # ตรวจสอบว่าเป็นกล่องข้อความหรือไม่
            if not getattr(shape, "has_text_frame", False):
                continue
            text = shape.text.strip().replace("\n", " ")
            if text:
                items.append((shape.top, shape.left, text))
        items.sort()
        combined = " ".join([t[2] for t in items]) + "\n\n"  # รวมข้อความในสไลด์
        pages.append((i, combined))
    return pages

# 📝 เขียนข้อความที่ดึงมาแล้วลงในไฟล์ .txt
def write_txt(lines, path):
    with open(path, "w", encoding="utf-8") as f:
        for i, text in lines:
            f.write(f"{i}-[{text}]\n")

# 🤖 สรุปข้อความจากแต่ละสไลด์โดยใช้ GPT (ภาษาอังกฤษ)
def summarise_with_gpt(lines, model="gpt-3.5-turbo"):
    openai.api_key = os.getenv("OPENAI_API_KEY")
    results = []
    for i, text in tqdm(lines):  # แสดง progress bar
        prompt = (
            "You are helping summarize a presentation designed for Thai researchers.\n"
            "The topic is about near-infrared spectral analysis (chemometrics).\n"
            "Please summarize the following slide content into one concise sentence in English, "
            "focusing on the key point this slide is trying to communicate:\n\n"
            f"{text}"
        )
        res = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant skilled in summarizing scientific slides."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=1000
        )
        summary = res.choices[0].message.content.strip()
        results.append((i, summary))
        time.sleep(1)  # รอ 1 วินาทีเพื่อหลีกเลี่ยง rate limit
    return results

# 🌐 แปลข้อความภาษาอังกฤษที่สรุปไว้ให้เป็นภาษาไทยโดยใช้ Google Translate
def translate_to_thai(lines):
    client = translate.Client()
    results = []
    for i, text in tqdm(lines):
        trans = client.translate(text, target_language="th")
        results.append((i, trans["translatedText"]))
    return results


## 🔐 การตั้งค่าคีย์ API และข้อมูลประจำตัว

ก่อนใช้งาน Google Translate API และ OpenAI GPT จำเป็นต้องตั้งค่าคีย์ API และไฟล์ข้อมูลประจำตัวให้เรียบร้อย

- `GOOGLE_APPLICATION_CREDENTIALS`: ใช้สำหรับกำหนดไฟล์ JSON ของบัญชีบริการ (Service Account) ของ Google Cloud
- `load_dotenv()`: โหลดค่าตัวแปรจากไฟล์ `.env` เช่น `OPENAI_API_KEY`

กรุณาใส่ path ของไฟล์ JSON และสร้างไฟล์ `.env` พร้อมกำหนดค่า API ที่จำเป็น
"""


In [2]:
# 🔑 ตั้งค่าตัวแปรสำหรับ Google Translate API โดยระบุ path ของไฟล์ Service Account JSON
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "insert json file here"

# 🔐 โหลดค่าตัวแปรสภาพแวดล้อมจากไฟล์ .env (ใช้สำหรับ API Key ของ OpenAI เป็นต้น)
load_dotenv()

True

## 📁 การตั้งค่าไฟล์อินพุตและเอาต์พุต

กำหนด path ของไฟล์ต่าง ๆ ที่ใช้ในกระบวนการทำงาน:

- `pptx_path`: ไฟล์ PowerPoint (.pptx) ต้นฉบับที่ต้องการประมวลผล
- `text_out`: ไฟล์ .txt สำหรับบันทึกข้อความที่ดึงมาจากแต่ละสไลด์
- `gpt_out`: ไฟล์ .txt สำหรับบันทึกสรุปภาษาอังกฤษจาก GPT
- `thai_out`: ไฟล์ .txt สำหรับบันทึกผลลัพธ์ที่แปลเป็นภาษาไทย

In [3]:
# 📥 กำหนด path ของไฟล์ PowerPoint ที่ต้องการประมวลผล (ตัวอย่าง: "example.pptx")
pptx_path = Path("2025_presen_Thai.pptx")

# 📝 ไฟล์เอาต์พุตสำหรับบันทึกข้อความที่ดึงมาจากสไลด์
text_out = Path("word_extract.txt")

# 📄 ไฟล์เอาต์พุตสำหรับบันทึกข้อความที่สรุปเป็นภาษาอังกฤษโดย GPT
gpt_out = Path("word_extract_gpt.txt")

# 🌐 ไฟล์เอาต์พุตสำหรับบันทึกข้อความที่แปลเป็นภาษาไทย
thai_out = Path("word_extract_gpt_th.txt")

## 🧾 การดึงข้อความจากสไลด์และบันทึกลงไฟล์

ขั้นตอนนี้จะ:

1. ดึงข้อความทั้งหมดจากสไลด์ในไฟล์ PowerPoint
2. บันทึกข้อความลงในไฟล์ `.txt` เพื่อใช้ในขั้นตอนถัดไป

In [4]:
# 📤 ดึงข้อความทั้งหมดจากไฟล์ PowerPoint ที่กำหนดไว้
pages = extract_slide_text(pptx_path)

# 💾 เขียนข้อความที่ได้จากแต่ละสไลด์ลงในไฟล์ .txt
write_txt(pages, text_out)

# ✅ แสดงข้อความยืนยันว่าบันทึกไฟล์สำเร็จ
print(f"{text_out} saved")

word_extract.txt saved


## 🧠 การสรุปเนื้อหาด้วย GPT

ขั้นตอนนี้จะใช้ GPT (OpenAI) เพื่อสรุปเนื้อหาจากแต่ละสไลด์  
ให้เป็นประโยคภาษาอังกฤษสั้น ๆ ที่สื่อสารใจความสำคัญของสไลด์นั้น

ผลลัพธ์จะถูกบันทึกลงในไฟล์ `.txt` เพื่อใช้งานต่อในขั้นตอนการแปล

In [5]:
# 🤖 ใช้ GPT สรุปข้อความของแต่ละสไลด์ให้อยู่ในรูปประโยคภาษาอังกฤษสั้น ๆ
summarised = summarise_with_gpt(pages)

# 💾 บันทึกผลลัพธ์ที่สรุปแล้วลงในไฟล์ .txt
write_txt(summarised, gpt_out)

# ✅ แสดงข้อความยืนยันว่าบันทึกไฟล์สำเร็จ
print(f"{gpt_out} saved")

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [06:23<00:00,  1.92s/it]

word_extract_gpt.txt saved


## 🌐 การแปลข้อความเป็นภาษาไทย

ในขั้นตอนนี้ ข้อความที่ถูกสรุปเป็นภาษาอังกฤษจะถูกส่งไปยัง Google Translate API  
เพื่อแปลเป็นภาษาไทย และบันทึกผลลัพธ์ลงในไฟล์ `.txt`
"""


In [ ]:
# 🌐 แปลข้อความภาษาอังกฤษที่สรุปไว้ให้เป็นภาษาไทยโดยใช้ Google Translate API
thai = translate_to_thai(summarised)

# 💾 บันทึกข้อความที่แปลแล้วลงในไฟล์ .txt
write_txt(thai, thai_out)

# ✅ แสดงข้อความยืนยันว่าบันทึกไฟล์สำเร็จ
print(f"{thai_out} saved")